In [ ]:
#encoding=utf8
import sys
import string
 
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import os
from gensim import corpora, models
import json
import pandas as pd
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from sklearn.datasets import load_svmlight_file

from sklearn import  svm
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import pandas 
import numpy as np
from collections import defaultdict
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
np.__config__.show()

In [3]:
bus_list = []
with open('/Volumes/data/yelp2016/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json') as data_file:    
    for line in data_file:
        bus_list.append(json.loads(line))

In [4]:
bus_list[0]

{'attributes': {'Accepts Credit Cards': True,
  'Alcohol': 'none',
  'Ambience': {'casual': False,
   'classy': False,
   'divey': False,
   'hipster': False,
   'intimate': False,
   'romantic': False,
   'touristy': False,
   'trendy': False,
   'upscale': False},
  'Attire': 'casual',
  'Caters': False,
  'Delivery': False,
  'Drive-Thru': False,
  'Good For': {'breakfast': False,
   'brunch': False,
   'dessert': False,
   'dinner': False,
   'latenight': False,
   'lunch': False},
  'Good For Groups': True,
  'Good for Kids': True,
  'Has TV': False,
  'Noise Level': 'average',
  'Outdoor Seating': False,
  'Parking': {'garage': False,
   'lot': False,
   'street': False,
   'valet': False,
   'validated': False},
  'Price Range': 1,
  'Take-out': True,
  'Takes Reservations': False,
  'Waiter Service': False},
 'business_id': '5UmKMjUEUNdYWqANhGckJw',
 'categories': ['Fast Food', 'Restaurants'],
 'city': 'Dravosburg',
 'full_address': '4734 Lebanon Church Rd\nDravosburg, PA 15034

In [5]:
business_df = pd.DataFrame.from_dict(bus_list)
business_df = business_df[['business_id','categories','stars']]
#( [category for categories in business_df['categories'] for category in categories] )

In [6]:
cuisine_df = business_df.apply(lambda x: pd.Series(x['categories']),axis=1).stack().reset_index(level=1, drop=True)

In [7]:
cuisine_df.name = 'cuisine'

In [8]:
business_df = business_df.drop('categories',axis=1).join(cuisine_df)

In [9]:
business_df.head()

,business_id,stars,cuisine
0,5UmKMjUEUNdYWqANhGckJw,3.5,Fast Food
0,5UmKMjUEUNdYWqANhGckJw,3.5,Restaurants
1,UsFtqoBl7naz8AVUBZMjQQ,3.0,Nightlife
2,cE27W9VPgO88Qxe4ol6y_g,2.5,Active Life
2,cE27W9VPgO88Qxe4ol6y_g,2.5,Mini Golf


In [10]:
#bus_list = []
#whole_dict = {}
#res_dict = {}
#smaller_dict = {}
#for line in open('/Volumes/data/yelp2016/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json')    :
#    temp_dict = {}    
#    temp_dict.update(json.loads(line))
#    busid = temp_dict["business_id"]
#    categories = temp_dict["categories"]
#    whole_dict[busid] = categories
#    for i in range(len(categories)):
#        if categories[i]=="Restaurants":
#            res_dict[busid]=categories[i]
#        if categories[i]=="Chinese" or categories[i]=="Italian" :
#            smaller_dict[busid]={"categories":categories[i]}
#
#print("size of whole busid:categorylist: " + str(len(whole_dict)))  
#print("number of restaurants: " + str(len(res_dict)))
#print("number of chinese or italian restaurants: " + str(len(smaller_dict)))



In [11]:
(business_df.head())
print('Number of businesses in dataset: ' + str(len(business_df['business_id'].unique())))
print('Number of restaurants: ' + str(len((business_df[(business_df.cuisine == 'Restaurants')]))))
print('Number of Italian or Chinese Restaurants: ' + 
      str(len((business_df[(business_df.cuisine == 'Italian') | (business_df.cuisine == 'Chinese')]))))
print('Number of Italian Restaurants: ' + str(len((business_df[(business_df.cuisine == 'Italian')]))))
print('Number of Chinese Restaurants: ' + str(len((business_df[(business_df.cuisine == 'Chinese')]))))

Number of businesses in dataset: 85901
Number of restaurants: 26729
Number of Italian or Chinese Restaurants: 3653
Number of Italian Restaurants: 1939
Number of Chinese Restaurants: 1714


In [12]:
from pprint import pprint
reviews = []
with open('/Volumes/data/yelp2016/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json') as data_file:    
    for line in data_file:
        reviews.append(json.loads(line))

In [13]:
(reviews[0])

{'business_id': '5UmKMjUEUNdYWqANhGckJw',
 'date': '2012-08-01',
 'review_id': 'Ya85v4eqdd6k9Od8HbQjyA',
 'stars': 4,
 'text': 'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.',
 'type': 'review',
 'user_id': 'PUFPaY9KxDAcGqfsorJp3Q',
 'votes': {'cool': 0, 'funny': 0, 'useful': 0}}

In [14]:
reviews_df = pd.DataFrame.from_dict(reviews)
reviews_df.head()

,business_id,date,review_id,stars,text,type,user_id,votes
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{'funny': 0, 'useful': 0, 'cool': 0}"
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,"{'funny': 0, 'useful': 0, 'cool': 0}"
2,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is a little out dated and not o...,review,auESFwWvW42h6alXgFxAXQ,"{'funny': 1, 'useful': 1, 'cool': 0}"
3,5UmKMjUEUNdYWqANhGckJw,2015-12-26,pVMIt0a_QsKtuDfWVfSk2A,3,PROS: Italian hoagie was delicious. Friendly ...,review,qiczib2fO_1VBG8IoCGvVg,"{'funny': 0, 'useful': 0, 'cool': 0}"
4,5UmKMjUEUNdYWqANhGckJw,2016-04-08,AEyiQ_Y44isJmNbMTyoMKQ,2,First the only reason this place could possibl...,review,qEE5EvV-f-s7yHC0Z4ydJQ,"{'funny': 0, 'useful': 1, 'cool': 0}"


In [15]:
reviews_df = reviews_df[['business_id','stars','text']]
reviews_df.head()

,business_id,stars,text
0,5UmKMjUEUNdYWqANhGckJw,4,"Mr Hoagie is an institution. Walking in, it do..."
1,5UmKMjUEUNdYWqANhGckJw,5,Excellent food. Superb customer service. I mis...
2,5UmKMjUEUNdYWqANhGckJw,5,Yes this place is a little out dated and not o...
3,5UmKMjUEUNdYWqANhGckJw,3,PROS: Italian hoagie was delicious. Friendly ...
4,5UmKMjUEUNdYWqANhGckJw,2,First the only reason this place could possibl...


In [16]:
reviews_df.dtypes

business_id    object
stars           int64
text           object
dtype: object

In [17]:
businessCI_df = business_df[(business_df.cuisine == 'Chinese') | (business_df.cuisine == 'Italian')]

In [18]:
final_df =  pd.merge(businessCI_df,reviews_df,on='business_id')
final_df.head(10)

,business_id,stars_x,cuisine,stars_y,text
0,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,3,Don Don is a restaurant I find myself missing ...
1,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,1,Take it from me; avoid this place at all cost....
2,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,4,Don Don is atleast as good as (although I pers...
3,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,Far away from real Chinese food. Doesn't even ...
4,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,I used to order takeout here once every month ...
5,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,4,Delicious Singapore noodles and seafood deligh...
6,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,3,I came here with two friends prior to visiting...
7,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,1,I use to order here fairly often. The past 2 ...
8,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,We visited Don Don Chinese restaurant in Carne...
9,P1fJb2WQ1mXoiudj8UE44w,3.5,Italian,5,I brought my husband and my parents all to Pap...


In [19]:
#final_df['cuisine'] = final_df['cuisine'].astype('category')
final_df['text'] = final_df['text'].str.lower()
final_df.dtypes

business_id     object
stars_x        float64
cuisine         object
stars_y          int64
text            object
dtype: object

In [20]:
final_df["code"] = np.where(final_df["cuisine"].str.contains("Italian"), 1,-1)

#pd.Categorical(final_df.cuisine,  ["Italian", "Chinese"], categories=[1, -1])
final_df.head(20)


,business_id,stars_x,cuisine,stars_y,text,code
0,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,3,don don is a restaurant i find myself missing ...,-1
1,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,1,take it from me; avoid this place at all cost....,-1
2,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,4,don don is atleast as good as (although i pers...,-1
3,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,far away from real chinese food. doesn't even ...,-1
4,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,i used to order takeout here once every month ...,-1
5,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,4,delicious singapore noodles and seafood deligh...,-1
6,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,3,i came here with two friends prior to visiting...,-1
7,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,1,i use to order here fairly often. the past 2 ...,-1
8,SQ0j7bgSTazkVQlF5AnqyQ,2.5,Chinese,2,we visited don don chinese restaurant in carne...,-1
9,P1fJb2WQ1mXoiudj8UE44w,3.5,Italian,5,i brought my husband and my parents all to pap...,1


In [21]:

final_df.dtypes

business_id     object
stars_x        float64
cuisine         object
stars_y          int64
text            object
code             int64
dtype: object

In [22]:
#review_dict = {}
#outcome_list = []
#for line in open('/Volumes/data/yelp2016/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'):
#    temp_dict = {}    
#    temp_dict.update(json.loads(line))
#    review_business = temp_dict["business_id"]
#    for business in smaller_dict.keys():
#        if review_business == business:
#            if review_business in review_dict:
#                review_dict[review_business]["categories"] = smaller_dict[business]["categories"]
#                review_dict[review_business]["revtext"].append(temp_dict["text"])
#                review_dict[review_business]["rating"].append(temp_dict["stars"])
#            else:
#                review_dict[review_business]={"categories":smaller_dict[business]["categories"],"revtext":[temp_dict["text"]],"rating":[temp_dict["stars"]]}
#print("number of businesses in review document: " + str(len(review_dict)))

In [23]:
tokenizer = RegexpTokenizer(r'\s+', gaps=True)

# create English stop words list
en_stop = set(get_stop_words('en'))
# print(en_stop)
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [24]:
#doc_set = []
#outcomes_list = []
#doc_outcome = []
#for k, v in review_dict.items():
#    for i in range(len(review_dict[k]["revtext"])):
#        if review_dict[k]["categories"] == "Chinese":
#            outcomes_list.append(-1)
#            doc_outcome.append((-1,review_dict[k]["revtext"][i]))
#        else:
#            outcomes_list.append(1)
#            doc_outcome.append((1,review_dict[k]["revtext"][i]))

In [25]:
doc_outcome = list(zip(final_df.code,final_df.text))
print("Total number of reviews: " + str(len(doc_outcome)))
doc_outcome[0]

Total number of reviews: 215908


(-1,
 "don don is a restaurant i find myself missing now and again, it's probably my favorite restaurant in carnegie. simple setup, just a small restaurant with a few tables and a tv. oddly enough i think my favorite thing to eat there is the wonton soup, never had better. the food is average but i always enjoyed eating there, great service and a cheap place to get chinese food.")

In [1]:
type(texts[1])

NameError: name 'texts' is not defined

In [ ]:
for i in range(len(texts)):
    print(texts[i])

In [ ]:
np.random(texts)

In [ ]:
#model_dict={}
#def create_learning_set(text_arr,set_size,model_name):
#    count_1 = 0
#    count_2 = 0
#    temp_outcome = []
#    for i in range(len(text_arr)):
#        if text_arr[i][0] == 1 and count_1 < set_size/2:
#            temp_outcome.append((text_arr[i][0],text_arr[i][1]))
#            count_1 = count_1 + 1
#        elif text_arr[i][0] == -1 and count_2 < set_size/2:
#            temp_outcome.append((text_arr[i][0],text_arr[i][1]))
#            count_2 = count_2 + 1
#    return(temp_outcome)
#
#outcomes_list_p, texts_learn_p = create_learning_set(texts,1000)
#outcomes_list_2p, texts_learn_2p = create_learning_set(texts,2000)
#outcomes_list_all, texts_learn_all = create_learning_set(texts,len(texts))


In [ ]:
def create_learning_set(text_arr, set_size, model_name):
    np.random(8)
    

In [ ]:
(x[1] for x in texts)

In [ ]:
dictionary = corpora.Dictionary([x[1] for x in texts])
print("there are: " + str(len(texts)) + " review documents in the dictionary")
print("dictionary done!")
os.system('say "Finished dictionary"')

In [ ]:
dictionary.values()

In [ ]:

def make_corpus(texts_tuple):
    text_list = [x[1] for x in texts_tuple]
    return  [dictionary.doc2bow(text) for text in text_list]


#corpus_p = make_corpus(texts_learn_p)
#corpus_2p = make_corpus(texts_learn_2p)
#corpus_all = make_corpus(texts_learn_all)


In [ ]:
def make_model_and_corpus(texts_tuple,num_top, num_pass, model_name):
    text_list = [x[1] for x in texts_tuple]
    temp_corpus = [dictionary.doc2bow(text) for text in text_list]
    print(model_name)
    temp_model=models.LdaMulticore(corpus=temp_corpus,num_topics=num_top,id2word=dictionary, passes=num_pass)
    temp_model_corpus = temp_model[temp_corpus]
    temp_outcomes = [x[0] for x in texts_tuple]
    model_dict[model_name] = {"model": temp_model,"model_corpus": temp_model_corpus, "outcomes":temp_outcomes}
    print(model_dict)
    #model_dict[model_name] = {"model": models.ldamodel.LdaModel(corpus_name, num_topics=num_top, id2word = dictionary, passes=pass_num)}
    
    #modelname.save("/Users/rungsunan/spyder/yelpproject/" + str(modelname))
    #print(modelname + " for " + corpus + " complete!")
       
#make_model_and_corpus(corpus_p,2,5,"lda_model_corpus_p")
#make_model_and_corpus(corpus_2p,2,5,"lda_model_corpus_2p")
#make_model_and_corpus(corpus_all,2,5,"lda_model_corpus_all")
def save_model_and_corpus(modelname):
    model_dict[modelname]['model'].save('/Users/rungsunan/'+ modelname + '.model')
    corpora.SvmLightCorpus.serialize('/Users/rungsunan/'+ modelname + '_corpus.svmlight', model_dict[modelname]['model_corpus'],labels=model_dict[modelname]['outcomes'])


def load_ldamodel(modelname):
    print(modelname)
    X_temp, y_temp = load_svmlight_file("/Users/rungsunan/" + modelname)
    return (X_temp,y_temp)


def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    clf = svm.SVC(kernel='linear')
    clf.fit(X, y)
    return clf

def histo_corpus(modelname):
    histo = []
    corpus = model_dict[modelname]['model_corpus']
    for doc in corpus:
        for i in range(len(doc)):
            if doc[i][1] > .1:
                histo.append(doc[i][0])
    return histo


def best_topics (texts_tuple):
    grid = defaultdict(list)
    param_list = []
    perplex_list = []
    perword_list = []
    text_list = [x[1] for x in texts_tuple]
    temp_corpus = [dictionary.doc2bow(text) for text in text_list]
    number_of_words = sum(cnt for document in temp_corpus for _, cnt in document)
    parameter_list = [2,3,4,5,10,15]
    for parameter_value in parameter_list:      
        print ("starting pass for parameter_value = %.3f" % parameter_value)
        model = models.LdaMulticore(corpus=temp_corpus, workers=None, id2word=dictionary, num_topics=parameter_value, passes=4, iterations=20)
        perplex = model.bound(temp_corpus) # this is model perplexity not the per word perplexity
        print ("Total Perplexity: %s" % perplex)
        param_list.append(parameter_value)
        perplex_list.append(perplex)
        grid[parameter_value].append(perplex)

    
        per_word_perplex = np.exp2(-perplex / number_of_words)
        perword_list.append(per_word_perplex)
        print ("Per-word Perplexity: %s" % per_word_perplex)
        grid[parameter_value].append(per_word_perplex)
        #model.save(data_path + 'ldaMulticore_i10_T' + str(parameter_value) + '_training_corpus.lda')
        print

    for numtopics in parameter_list:
        print (numtopics, '\t',  grid[numtopics])
    df = pandas.DataFrame(grid)
    ax = plt.figure(figsize=(7, 4), dpi=300).add_subplot(111)
    df.iloc[1].transpose().plot(ax=ax,  color="#254F09")
    plt.xlim(parameter_list[0], parameter_list[-1])
    plt.ylabel('Perplexity')
    plt.xlabel('topics')
    plt.title('')
    plt.show()
    #df.to_pickle(data_path + 'gensim_multicore_i10_topic_perplexity.df')


In [ ]:
texts_outcome_p100k = create_learning_set(texts,100000,"lda_model_corpus_100k")

In [ ]:
texts_outcome_p2 = create_learning_set(texts,1000,"lda_model_corpus_p2")
texts_outcome_p = create_learning_set(texts,3000,"lda_model_corpus_p")
texts_outcome_2p = create_learning_set(texts,7000,"lda_model_corpus_2p")
texts_outcome_p10k = create_learning_set(texts,10000,"lda_model_corpus_2p")
texts_outcome_p100k = create_learning_set(texts,100000,"lda_model_corpus_100k")
texts_outcome_all = create_learning_set(texts,len(texts),"lda_model_corpus_all")

In [ ]:
(texts_outcome_p100k[0])

In [ ]:
best_topics(texts_outcome_p100k)

In [ ]:
best_topics(texts_outcome_all)
best_topics(texts_outcome_p2)
best_topics(texts_outcome_p)
best_topics(texts_outcome_2p)
best_topics(texts_outcome_p10k)

In [ ]:
corpus_p100k = make_corpus(texts_outcome_p100k)

In [ ]:
corpus_all = make_corpus(texts_outcome_all)
corpus_p10k = make_corpus(texts_outcome_p10k)
corpus_p2 = make_corpus(texts_outcome_p2)

In [ ]:
make_model_and_corpus(texts_outcome_p100k,5  ,10,"lda_model_p100k_5topics")

In [ ]:
make_model_and_corpus(texts_outcome_all,2  ,20,"lda_model_all_2topics")
#make_model_and_corpus(texts_outcome_p10k,2  ,20,"lda_model_p10k_2topics")
#make_model_and_corpus(texts_outcome_p10k,3  ,20,"lda_model_p10k_3topics")
#make_model_and_corpus(texts_outcome_p10k,10  ,20,"lda_model_p10k_10topics")

In [ ]:
save_model_and_corpus('lda_model_p100k_5topics')
#save_model_and_corpus('lda_model_p10k_2topics')
#save_model_and_corpus('lda_model_p10k_3topics')
#save_model_and_corpus('lda_model_p10k_10topics')


In [ ]:
#(X_all_2topics, y_all_2topics) = load_ldamodel("lda_model_all_2topics_corpus.svmlight")
(X_p100k_5topics, y_p100k_5topics) = load_ldamodel("lda_model_p100k_5topics_corpus.svmlight")
#(X_p10k_3topics, y_p10k_3topics) = load_ldamodel("lda_model_p10k_3topics_corpus.svmlight")
#(X_p10k_10topics, y_p10k_10topics) = load_ldamodel("lda_model_p10k_10topics_corpus.svmlight")


In [ ]:
#X_train_all_2topics, X_test_all_2topics, y_train_all_2topics, y_test_all_2topics = train_test_split(
#  X_all_2topics, y_all_2topics, test_size=0.2, random_state=0)

X_train_p100k_5topics, X_test_p100k_5topics, y_train_p100k_5topics, y_test_p100k_5topics = train_test_split(
  X_p100k_5topics, y_p100k_5topics, test_size=0.2, random_state=0)
#X_train_p10k_3topics, X_test_p10k_3topics, y_train_p10k_3topics, y_test_p10k_3topics = train_test_split(
#  X_p10k_3topics, y_p10k_3topics, test_size=0.2, random_state=0)
#X_train_p10k_10topics, X_test_p10k_10topics, y_train_p10k_10topics, y_test_p10k_10topics = train_test_split(
#  X_p10k_10topics, y_p10k_10topics, test_size=0.2, random_state=0)


In [ ]:
np.shape(X_train_p100k_5topics)

In [ ]:
X_train_p100k_5topics.todense()[0]

In [ ]:
grid.fit(p)

In [ ]:
k_range = range(1,10)
param_grid = dict(n_neighbors=k_range)
knn=KNeighborsClassifier()
grid=GridSearchCV(knn,param_grid,cv=10,scoring='accuracy',n_jobs=1)

In [ ]:
grid.fit(X_p100k_5topics,y_p100k_5topics)

In [ ]:
#grid.fit(X_p10k_2topics,y_p10k_2topics)
grid.grid_scores_
grid.cv_results_['mean_test_score'][0]

In [ ]:
s = [result.mean_validation_score for result in grid.grid_scores_]
print(grid.cv_results_['mean_test_score'][0])
plt.plot(k_range,grid.cv_results_['mean_test_score'][0])
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
tuned_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
scores = ['precision','recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=10,
                       scoring='%s_macro' % score,n_jobs=1, verbose=100)
    clf.fit(X_train_p100k_5topics, y_train_p100k_5topics)
    #    clf.fit(X_train_p10k_2topics, y_train_p10k_2topics)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true_p100k_5topics, y_pred_p100k_5topics = y_test_p100k_5topics, clf.predict(X_test_p100k_5topics)
    print(classification_report(y_true_p100k_5topics, y_pred_p100k_5topics))
    print(confusion_matrix(y_true_p100k_5topics, y_pred_p100k_5topics))
    print()
    os.system('say "doc set done"')

In [ ]:
k_range = range(1,25)
param_grid = dict(n_neighbors=k_range)
knn=KNeighborsClassifier()
grid=GridSearchCV(knn,param_grid,cv=10,scoring='accuracy',n_jobs=-1)
grid.fit(X_p10k_3topics,y_p10k_3topics)
grid.grid_scores_
grid_mean_scores = [result.mean_validation_score for result in grid.grid_scores_]
print(grid_mean_scores)
plt.plot(k_range,grid_mean_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
tuned_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10,100,1000]}
scores = ['precision','recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=10,
                       scoring='%s_macro' % score,n_jobs=-1)
    clf.fit(X_train_p10k_3topics, y_train_p10k_3topics)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true_p10k_3topics, y_pred_p10k_3topics = y_test_p10k_3topics, clf.predict(X_test_p10k_3topics)
    print(classification_report(y_true_p10k_3topics, y_pred_p10k_3topics))
    print(confusion_matrix(y_true_p10k_3topics, y_pred_p10k_3topics))
    print()
    os.system('say "doc set done"')


In [ ]:
k_range = range(1,25)
param_grid = dict(n_neighbors=k_range)
knn=KNeighborsClassifier()
grid=GridSearchCV(knn,param_grid,cv=10,scoring='accuracy',n_jobs=-1)
grid.fit(X_p10k_10topics,y_p10k_10topics)
grid.grid_scores_
grid_mean_scores = [result.mean_validation_score for result in grid.grid_scores_]
print(grid_mean_scores)
plt.plot(k_range,grid_mean_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
tuned_parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10,100,1000]}
scores = ['precision','recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=10,
                       scoring='%s_macro' % score,n_jobs=-1)
    clf.fit(X_train_p10k_10topics, y_train_p10k_10topics)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true_p10k_10topics, y_pred_p10k_10topics = y_test_p10k_10topics, clf.predict(X_test_p10k_10topics)
    print(classification_report(y_true_p10k_10topics, y_pred_p10k_10topics))
    print(confusion_matrix(y_true_p10k_10topics, y_pred_p10k_10topics))
    print()
    os.system('say "doc set done"')

In [ ]:
histo_2p = histo_corpus("lda_model_p2")
plt.hist(histo_2p)

In [ ]:
globals()